<a href="https://colab.research.google.com/github/ArchitGupta16/Topic-Modeling/blob/main/Pipeline_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Data And Packakges
from sklearn.datasets import fetch_20newsgroups
# cats = ['sci.space']
data = fetch_20newsgroups(subset='all')['data']

In [ ]:
print(data[29])

In [ ]:
len(data)

18846

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 128.1 MB/s eta 0:00:00


In [ ]:
#@title Summarization
model_name = "t5-base"
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from tqdm import tqdm


In [ ]:
final= [None]*len(data)

In [ ]:
copy = [None]*len(data)
for i in range(0,len(data)):
  copy[i] = data[i].split(",From")


In [ ]:
for i in tqdm(range(len(copy))):
  tex=""
  if type(copy[i]) == list:
    tex = ''.join([str(elem) for elem in copy[i]])
  else:
    tex = copy[i]
  input_ids = tokenizer.encode("summarize: " + tex, return_tensors="pt", add_special_tokens=True)
  generated_ids = model.generate(input_ids=input_ids,num_beams=5,max_length=100,repetition_penalty=2.5,length_penalty=1,early_stopping=True,num_return_sequences=1)
  preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
  final[i]=preds
  tex = ""


In [ ]:
flat_list = []
for sublist in final:
    for item in sublist:
        flat_list.append(item)

In [ ]:
#@title Drive - for uploading summary
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
writeBook = open("/content/gdrive/MyDrive/Colab Notebooks/R&D/data.txt", "a")

In [ ]:
# cont = open("{}.txt".format('data'), "a")
for i in range(len(flat_list)):
  writeBook.write(flat_list[i]+"\n")

In [ ]:
writeBook.close()
# writeBook = open("{}.txt".format('data'), "w+")

In [ ]:
#@title Summary Content from Drive
inp = open("/content/gdrive/MyDrive/Colab Notebooks/R&D/0_11000.txt", "r")


In [ ]:
inp

<_io.TextIOWrapper name='/content/gdrive/MyDrive/Colab Notebooks/R&D/0_11000.txt' mode='r' encoding='UTF-8'>

In [ ]:
l = []

In [ ]:
for line in inp.readlines():
  l.append(line)
l = l[:1000]
l
print(len(l))

1000


In [ ]:
data = copy

In [ ]:
#@title Library
# !python -m pip install Pytorch
# !python -m pip install SentenceTransformer
!pip install -U sentence-transformers
!pip install hdbscan

In [ ]:
#@title Embeddings
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(l, show_progress_bar=True)
print(embeddings)

In [ ]:
!pip install umap-learn

In [ ]:
#@title UMAP
import umap.umap_ as um
umap_embeddings = um.UMAP(n_neighbors=15, n_components=2, metric='cosine').fit_transform(embeddings)


In [ ]:
#@title HDBSCAN
import hdbscan
cluster = hdbscan.HDBSCAN(min_samples=5,min_cluster_size=75,metric='euclidean', cluster_selection_method='leaf').fit(umap_embeddings)

In [ ]:
import matplotlib.pyplot as plt
import umap.umap_ as umap
import pandas as pd
# Prepare data
umap_data = umap.UMAP(n_neighbors=15, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
result = pd.DataFrame(umap_data, columns=['x', 'y'])
result['labels'] = cluster.labels_

# Visualize clusters
fig, ax = plt.subplots(figsize=(20, 10))
outliers = result.loc[result.labels == -1, :]
clustered = result.loc[result.labels != -1, :]
plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.05)
plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.05, cmap='hsv_r')
plt.colorbar()

In [ ]:
#@title c TF-IDF
docs_df = pd.DataFrame(l, columns=["Doc"])
docs_df['Topic'] = cluster.labels_
docs_df['Doc_ID'] = range(len(docs_df))
docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(1, 1)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count

tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m=len(l))

In [ ]:
#@title Topic Representation
def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names_out()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic']).Doc.count().reset_index().rename({"Topic": "Topic", "Doc": "Size"}, axis='columns').sort_values("Size", ascending=False))
    return topic_sizes

top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)
topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

In [ ]:
top_n_words[0][:10]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
for i in range(20):
    # Calculate cosine similarity
    similarities = cosine_similarity(tf_idf.T)
    np.fill_diagonal(similarities, 0)

    # Extract label to merge into and from where
    topic_sizes = docs_df.groupby(['Topic']).count().sort_values("Doc", ascending=False).reset_index()
    topic_to_merge = topic_sizes.iloc[-1].Topic
    topic_to_merge_into = np.argmax(similarities[topic_to_merge + 1]) - 1

    # Adjust topics
    docs_df.loc[docs_df.Topic == topic_to_merge, "Topic"] = topic_to_merge_into
    old_topics = docs_df.sort_values("Topic").Topic.unique()
    map_topics = {old_topic: index - 1 for index, old_topic in enumerate(old_topics)}
    docs_df.Topic = docs_df.Topic.map(map_topics)
    docs_per_topic = docs_df.groupby(['Topic'], as_index = False).agg({'Doc': ' '.join})

    # Calculate new topic words
    m = len(l)
    tf_idf, count = c_tf_idf(docs_per_topic.Doc.values, m)
    top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20)

topic_sizes = extract_topic_sizes(docs_df); topic_sizes.head(10)

In [ ]:
top_n_words[4][:10]

[('year', 0.03776856683976911),
 ('clutch', 0.02979047171254766),
 ('days', 0.027647128635489175),
 ('90', 0.026833018928390996),
 ('asbestos', 0.025758058809400554),
 ('car', 0.025547778936764954),
 ('homicides', 0.024744498953434207),
 ('handgun', 0.022772863763996433),
 ('left', 0.021127821375493384),
 ('10', 0.01944761259441615)]

In [ ]:
len(top_n_words)

In [ ]:
#@title Pre-processing Coherence
tops = []
l = []
for i in top_n_words.values():
  for j in i:
    l.append(j[0])
  tops.append(l)
  l = []

In [ ]:
#@title Coherence
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_20newsgroups

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

In [ ]:
texts, _ = fetch_20newsgroups( subset='all',return_X_y=True )
tokenizer = lambda s: re.findall( '\w+', s.lower() )
texts = [ tokenizer(t) for t in  texts ]

In [ ]:
# Creating a dictionary with the vocabulary
word2id = Dictionary( texts )

# Coherence model
cm = CoherenceModel(topics=tops, texts=texts, coherence='c_v', dictionary=word2id)
coherence_per_topic = cm.get_coherence_per_topic()

In [ ]:
coherence_per_topic

In [ ]:
sum(coherence_per_topic)/len(coherence_per_topic)

In [ ]:
#@title Pre-processing Perplexity
l = []
perplex = []
count = 0
for i in top_n_words.values():
  for j in i:
    l.append(j[0])
  perplex.append(l)
  count+=1
  l = []

In [ ]:
#@title Perplexity
!pip install nltk

In [ ]:
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_20newsgroups

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

import collections, nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
corpus, _ = fetch_20newsgroups( subset='all',return_X_y=True )
print(type(corpus))
tokenizer = lambda s: re.findall( '\w+', s.lower() )
tokens = [ tokenizer(t) for t in  corpus ]

flat = []
for element in tokens:
    if type(element) is list:
        for item in element:
            flat.append(item)
    else:
        flat.append(element)

tokens = flat

In [ ]:
def unigram(tokens):
    model = collections.defaultdict(lambda: 0.01)
    for f in tokens:
        try:
            model[f] += 1
        except KeyError:
            model [f] = 1
            continue
    N = float(sum(model.values()))
    for word in model:
        model[word] = model[word]/N
    return model

In [ ]:
#computes perplexity of the unigram model on a testset
def perplexity(testset, model):
    testset = testset.split()
    perplexity = 1
    N = 0
    for word in testset:
        N += 1
        perplexity = perplexity * (1/model[word])
    perplexity = pow(perplexity, 1/float(N))
    return perplexity


model = unigram(tokens)
dic = {}
# for i in perplex:
  # for j in i:
  #   dic[j] = perplexity(j,model)
count=0
for i in perplex:
  d = {}
  count += 1
  for j in i:
    d[j] = None
  dic[count] = d
print(dic)


for i in dic.values():
  print(i)
  for key, value in i.items():
    print(key,value)
    i[key] = perplexity(key,model)

In [ ]:
dic